In [1]:
# import visualization packages
from polygon import RESTClient
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px

# import pandas and datetime
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt
import time
import warnings
warnings.filterwarnings('ignore')

# import API_KEY from config
from config import *

In [2]:
# Enter desire ticker
ticker = 'AAPL'

# Find valid trading dates

In [3]:
# get the date that is 30 days before expiration
earnings_date = '2022-10-28'
expiration_date = '2022-11-04'
start_date = (pd.to_datetime(earnings_date) - dt.timedelta(days=30)).date().strftime('%Y-%m-%d')

In [4]:
nyse = mcal.get_calendar('NYSE')

In [5]:
valid_dates = nyse.valid_days(start_date=start_date, end_date=earnings_date)
valid_dates = [date.date().strftime('%Y-%m-%d') for date in valid_dates]

In [6]:
client = RESTClient(API_KEY)

In [7]:
def get_net_pnl(call_df, put_df, start_date):
    call = client.get_aggs(
        ticker = call_df['option ticker'].iloc[0],
        limit=50000,
        multiplier = 1, 
        timespan = 'day', 
        from_ = start_date, 
        to = earnings_date)
    put = client.get_aggs(
        ticker = put_df['option ticker'].iloc[0],
        limit=50000,
        multiplier = 1, 
        timespan = 'day', 
        from_ = start_date, 
        to = earnings_date)
    call = pd.DataFrame(call)
    call['date'] = pd.to_datetime(call['timestamp'] * 1000000)

    put = pd.DataFrame(put)
    put['date'] = pd.to_datetime(put['timestamp'] * 1000000)
    
    # create dataframe of net close of straddle
    net_df = pd.DataFrame()
    net_df['close'] = call['close'] + put['close']
    
    # creates column name based on how many days before earnings the straddle was placed
    col_name = f'{len(net_df)} days'
    net_df[col_name] = round(((net_df['close'] - net_df['close'].iloc[0]) / net_df['close'].iloc[0]) * 100, 2)
    
    # just copy the date column from call and use it for the net_df
    net_df['date'] = call['date']
    
    # create a 'days remaining' before earnings column
#     net_df['days remaining'] = sorted(range(len(net_df)), reverse=True)
    
    net_df.drop('close', axis=1, inplace=True)
    return net_df

In [8]:
contracts_df = []
for c in client.list_options_contracts(ticker, limit=1000, as_of=earnings_date):
    contracts_df.append({
        'expiration date': c.expiration_date,
        'type': c.contract_type,
        'strike price': c.strike_price,
        'option ticker': c.ticker
    })
contracts_df = pd.DataFrame(contracts_df)
contracts_df['expiration date'] = pd.to_datetime(contracts_df['expiration date'])

In [9]:
# get the price history of the underlying ticker from the start_date to earnings_date 
underlying = client.get_aggs(ticker=ticker, from_=start_date, to=earnings_date, multiplier=1, timespan='day')
underlying = pd.DataFrame(underlying)
underlying['date'] = pd.to_datetime(underlying['timestamp'] * 1000000).dt.date

In [10]:
expiration_mask = contracts_df['expiration date'] == expiration_date
call_mask = contracts_df['type'] == 'call'
put_mask = contracts_df['type'] == 'put'

In [11]:
list_of_df = []
for date in valid_dates:
    start_date = date
    
    start_date_mask = underlying['date'] == pd.to_datetime(start_date)
    stock_price = underlying[start_date_mask]['close']
    contracts_df.iloc[abs(contracts_df[expiration_mask]['strike price'] - stock_price.iloc[0]).sort_values().head(2).index]
    
    call = contracts_df.iloc[abs(contracts_df[expiration_mask & call_mask]['strike price'] - stock_price.iloc[0]).sort_values().head(1).index]
    put = contracts_df.iloc[abs(contracts_df[expiration_mask & put_mask]['strike price'] - stock_price.iloc[0]).sort_values().head(1).index]
    
    list_of_df.append(get_net_pnl(call, put, start_date))
#     time.sleep(1)

In [45]:
all_df = pd.DataFrame(columns=['date'])
for dframe in list_of_df:
    all_df = pd.merge(all_df, dframe, how='outer', on='date')

# use this to make 'days remaining' the x-axis
all_df['days remaining'] = sorted(range(len(all_df)), reverse=True)
all_df['days remaining'] = all_df['days remaining'].astype('str') 
all_df['days remaining'].iloc[-1] = 'After Earnings'
all_df.set_index('days remaining', inplace=True)
all_df.drop('date', axis=1, inplace=True)

# use code below to make date the x-axis instead
# all_df.set_index('date', inplace=True)

In [44]:
fig = px.imshow(all_df.transpose(), color_continuous_scale=[(0,'red'), (0.5,'white'), (1.0, 'green')], range_color=(-100 ,100), text_auto=True)
fig.update_layout(
    title='Apple ATM Straddle Performance Leading up to Earnings in percent',
    yaxis_title='Straddle Initiated'
)
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]),  # hide weekends, eg. hide sat to before mon])
fig.show()

plot(fig, auto_open=True)

'temp-plot.html'